# ImmuneMethylTools — Validation Notebook
## Data Cleaning to Discovery: A Step-by-Step Walkthrough

---

Purpose: This notebook is the analyst's guided tour of the ImmuneMethylTools pipeline. Each step exposes a real pitfall in B-cell/T-cell DNA methylation analysis, applies a targeted fix from the `core/` library, and shows exactly what changes.

Design rule: All detection logic, thresholds, and statistical tests are imported from `core/`. Notebook cells contain only data access, display calls, and matplotlib/seaborn visualization.

| Step | Focus | Key Artifact Addressed |
|------|-------|------------------------|
| 0    | Raw EDA          | All 6 stumper artifacts; batch confound |
| 1    | Identity & Integrity | Sex metadata mixup; technical duplicates |
| 2    | Surgical Strike  | Clonal VDJ amplification artifact |
| 3    | Harmonization    | Batch × disease confound |
| 4    | Scientific Discovery | True biological DMR signal |
| 5    | ML Validation    | Honest vs. inflated AUC |


In [ ]:
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# ── Core module imports (all logic lives here) ──────────────────────────────
from core.io_utils import data_path, load_methylation
from core.qc_guard import audit_quality, detect_contamination, filter_site_quality
from core.xci_guard import compute_xci_signal, detect_sex_mixups
from core.sample_audit import detect_duplicates
from core.normalizer import check_confounding, robust_normalize
from core.repertoire_clonality import flag_clonal_artifacts, mask_clonal_vdj_sites
from core.dmr_hunter import find_dmrs
from core.ml_guard import run_safe_model

%matplotlib inline
plt.rcParams.update({
    'figure.dpi': 120,
    'font.size': 11,
    'axes.spines.top': False,
    'axes.spines.right': False,
})

# Shared palettes
DISEASE_PAL  = {'Case': '#E05C5C', 'Control': '#4A90D9'}
BATCH_PAL    = {'Batch_01': '#F5A623', 'Batch_02': '#7ED321', 'Batch_03': '#9B59B6'}
SEX_PAL      = {'M': '#3498DB', 'F': '#E91E63'}

# ── Inline PCA helper (sklearn only, not reimplementing math) ───────────────
def _run_pca(df, value_col='beta_value'):
    pivot = df.pivot_table(index='sample_id', columns='cpg_id', values=value_col)
    pivot = pivot.fillna(pivot.median())
    scaled = StandardScaler().fit_transform(pivot)
    pca    = PCA(n_components=2, random_state=42)
    coords = pca.fit_transform(scaled)
    result = pd.DataFrame(coords, columns=['PC1', 'PC2'], index=pivot.index).reset_index()
    meta   = df[['sample_id', 'batch_id', 'disease_label']].drop_duplicates('sample_id')
    return result.merge(meta, on='sample_id'), pca.explained_variance_ratio_

print('All core modules loaded. Ready to begin the walkthrough.')


---
## Step 0: The "Broken" Data — Raw Exploratory Data Analysis

We load `mock_methylation.csv` without any filtering. The raw data contains seven deliberately injected artifacts plus a true biological signal. Our job is to find and fix each one before calling a single DMR.


In [ ]:
# ── Load raw data ──────────────────────────────────────────────────────────
CSV_PATH = data_path('mock_methylation.csv')
df_raw   = load_methylation(CSV_PATH, verbose=False)

n_samples  = df_raw['sample_id'].nunique()
n_cpgs     = df_raw['cpg_id'].nunique()

print(f'Shape        : {len(df_raw):,} rows  x  {len(df_raw.columns)} columns')
print(f'Samples      : {n_samples}')
print(f'CpG sites    : {n_cpgs:,}')
print()

meta = df_raw.drop_duplicates('sample_id')[['sample_id', 'patient_id', 'batch_id', 'disease_label', 'sex']]
print('Disease breakdown:')
print(meta['disease_label'].value_counts().to_string())
print()
print('Batch breakdown:')
print(meta['batch_id'].value_counts().to_string())
print()
print('First 5 rows (long format):')
display(df_raw.head())


### Visual 1 — The Six Stumper Artifacts

Each panel exposes one artifact visible in the raw data. These are exactly the patterns that would mislead a naive pipeline into reporting false positives.


In [ ]:
# ── Precompute per-sample stats and XCI signal ──────────────────────────────
sample_stats = (
    df_raw.groupby('sample_id')
    .agg(mean_ncpg=('non_cpg_meth_rate', 'mean'),
         mean_depth=('depth', 'mean'),
         disease=('disease_label', 'first'),
         batch=('batch_id', 'first'),
         sex=('sex', 'first'))
    .reset_index()
    .sort_values('sample_id')
    .reset_index(drop=True)
)

# XCI signal from core module
xci_signal = compute_xci_signal(df_raw)
sex_flagged_ids, _ = detect_sex_mixups(df_raw)
xci_signal['mismatch'] = xci_signal['sample_id'].isin(sex_flagged_ids)
xci_signal = xci_signal.reset_index(drop=True)

# VDJ rows
vdj_df  = df_raw[df_raw['is_vdj_region']].copy()
nvdj_df = df_raw[~df_raw['is_vdj_region']].sample(600, random_state=42)

# Duplicate pair
s010 = df_raw[df_raw['sample_id'] == 'S010'][['cpg_id', 'beta_value']].rename(columns={'beta_value': 'S010'})
sdup = df_raw[df_raw['sample_id'] == 'S_DUP'][['cpg_id', 'beta_value']].rename(columns={'beta_value': 'S_DUP'})
dup_merged = s010.merge(sdup, on='cpg_id')

# ── 6-panel figure ─────────────────────────────────────────────────────────
fig, axes = plt.subplots(2, 3, figsize=(18, 10))
fig.suptitle('The Six Stumper Artifacts — Raw Data', fontsize=14, fontweight='bold', y=1.01)

x_idx = range(len(sample_stats))

# ── Panel 1: Bisulfite Conversion Failure ──────────────────────────────────
ax = axes[0, 0]
colors_bs = ['#E05C5C' if r > 0.02 else '#4A90D9' for r in sample_stats['mean_ncpg']]
ax.bar(x_idx, sample_stats['mean_ncpg'], color=colors_bs, alpha=0.85, edgecolor='white', linewidth=0.4)
ax.axhline(0.02, color='red', linestyle='--', linewidth=1.5, label='Fail threshold (2%)')
n_bs_fail = (sample_stats['mean_ncpg'] > 0.02).sum()
ax.text(0.97, 0.95, f'{n_bs_fail} samples failed', transform=ax.transAxes,
        ha='right', va='top', color='#E05C5C', fontsize=10, fontweight='bold')
ax.set_xlabel('Sample index')
ax.set_ylabel('Mean non-CpG meth. rate')
ax.set_title('Artifact 1 — Bisulfite Conversion\nFailure (non-CpG rate > 2%)', fontweight='bold')
ax.legend(fontsize=9)

# ── Panel 2: Contamination — bimodal KDE ──────────────────────────────────
ax = axes[0, 1]
beta_contaminated = df_raw[df_raw['sample_id'] == 'S020']['beta_value']
beta_normal       = df_raw[df_raw['sample_id'] == 'S005']['beta_value']
beta_contaminated.plot.kde(ax=ax, color='#E05C5C', linewidth=2.5, label='S020 (contaminated)')
beta_normal.plot.kde(ax=ax, color='#4A90D9', linewidth=2.5, label='S005 (normal)')
ax.axvspan(0.35, 0.65, alpha=0.12, color='orange', label='Bimodal "muddy" zone')
ax.set_xlabel('Beta value')
ax.set_ylabel('Density')
ax.set_title('Artifact 2 — Sample Contamination\n"Muddy" bimodal beta distribution', fontweight='bold')
ax.legend(fontsize=8)

# ── Panel 3: Low Coverage ──────────────────────────────────────────────────
ax = axes[0, 2]
colors_dep = ['#E05C5C' if d < 10 else '#4A90D9' for d in sample_stats['mean_depth']]
ax.bar(x_idx, sample_stats['mean_depth'], color=colors_dep, alpha=0.85, edgecolor='white', linewidth=0.4)
ax.axhline(10, color='red', linestyle='--', linewidth=1.5, label='Depth threshold (10x)')
n_dep_fail = (sample_stats['mean_depth'] < 10).sum()
ax.text(0.97, 0.95, f'{n_dep_fail} samples below threshold',
        transform=ax.transAxes, ha='right', va='top', color='#E05C5C', fontsize=9, fontweight='bold')
ax.set_xlabel('Sample index')
ax.set_ylabel('Mean read depth (x)')
ax.set_title('Artifact 3 — Low Coverage\nMean depth < 10x fails QC', fontweight='bold')
ax.legend(fontsize=9)

# ── Panel 4: Clonal VDJ Artifact ──────────────────────────────────────────
ax = axes[1, 0]
ax.scatter(nvdj_df['fragment_length'], nvdj_df['beta_value'],
           alpha=0.15, s=8, color='#AAAAAA', label='Non-VDJ sites (sample)')
ax.scatter(vdj_df['fragment_length'], vdj_df['beta_value'],
           alpha=0.55, s=14, color='#E05C5C', label='VDJ sites')
ax.axhline(0.80, color='#8B0000', linestyle='--', linewidth=1.2, alpha=0.8)
ax.axvline(180,  color='#8B0000', linestyle='--', linewidth=1.2, alpha=0.8)
ax.text(183, 0.82, 'Clonal zone\n(beta>0.8, frag>180bp)', color='#8B0000', fontsize=8)
ax.set_xlabel('Fragment length (bp)')
ax.set_ylabel('Beta value')
ax.set_title('Artifact 4 — Clonal VDJ Artifact\n"Fake Biomarker": high beta + long fragment', fontweight='bold')
ax.legend(fontsize=8)

# ── Panel 5: Sex Metadata Mixup ────────────────────────────────────────────
ax = axes[1, 1]
for sex_label in ['M', 'F']:
    grp = xci_signal[xci_signal['sex'] == sex_label]
    ax.scatter(grp.index, grp['mean_x_beta'],
               color=SEX_PAL[sex_label], alpha=0.65, s=40, zorder=2,
               label=f'Reported {sex_label}')
mismatched = xci_signal[xci_signal['mismatch']]
ax.scatter(mismatched.index, mismatched['mean_x_beta'],
           s=150, marker='x', color='red', zorder=5, linewidths=2.5,
           label=f'Mismatch flagged (n={len(mismatched)})')
ax.axhspan(0.40, 0.65, alpha=0.08, color='#E91E63', label='Expected female range')
ax.axhspan(0.00, 0.35, alpha=0.08, color='#3498DB', label='Expected male range')
ax.set_xlabel('Sample index (all samples)')
ax.set_ylabel('Mean X-chromosome beta')
ax.set_title('Artifact 5 — Sex Metadata Mixup\nX-linked methylation contradicts reported sex', fontweight='bold')
ax.legend(fontsize=7, ncol=2)

# ── Panel 6: Technical Duplicate ──────────────────────────────────────────
ax = axes[1, 2]
sample_pts = dup_merged.sample(min(500, len(dup_merged)), random_state=42)
ax.scatter(sample_pts['S010'], sample_pts['S_DUP'], alpha=0.4, s=10, color='#9B59B6')
r_val = dup_merged[['S010', 'S_DUP']].corr().iloc[0, 1]
ax.plot([0, 1], [0, 1], 'r--', linewidth=1.5, label=f'Identity line (r = {r_val:.4f})')
ax.set_xlim(0, 1); ax.set_ylim(0, 1)
ax.set_xlabel('S010 beta value')
ax.set_ylabel('S_DUP beta value')
ax.set_title(f'Artifact 6 — Technical Duplicate\nS010 vs S_DUP  (Pearson r = {r_val:.4f})', fontweight='bold')
ax.legend(fontsize=9)

plt.tight_layout()
plt.show()


### Visual 2 — The Batch Confound: Why a Naive Analysis Fails

PCA on the raw data reveals that samples cluster by sequencing batch rather than by disease label. `Batch_01` was enriched with Case samples during experiment setup, so any naive differential methylation test would report batch-driven differences as biological signal.


In [ ]:
pca_raw, var_raw = _run_pca(df_raw, value_col='beta_value')

fig, axes = plt.subplots(1, 2, figsize=(14, 6))
fig.suptitle('PCA on Raw Data — The Batch Confound', fontsize=13, fontweight='bold')

# Left: color by batch
ax = axes[0]
for batch, grp in pca_raw.groupby('batch_id'):
    ax.scatter(grp['PC1'], grp['PC2'], label=batch,
               color=BATCH_PAL.get(batch, 'gray'), alpha=0.85, s=70, edgecolors='white', linewidths=0.5)
ax.set_xlabel(f'PC1 ({var_raw[0]:.1%} variance)')
ax.set_ylabel(f'PC2 ({var_raw[1]:.1%} variance)')
ax.set_title('Colored by Batch\nSamples cluster by plate, not biology')
ax.legend(title='Batch', fontsize=9)

# Right: color by disease — clusters don't match
ax = axes[1]
for disease, grp in pca_raw.groupby('disease_label'):
    ax.scatter(grp['PC1'], grp['PC2'], label=disease,
               color=DISEASE_PAL.get(disease, 'gray'), alpha=0.85, s=70, edgecolors='white', linewidths=0.5)
ax.set_xlabel(f'PC1 ({var_raw[0]:.1%} variance)')
ax.set_ylabel(f'PC2 ({var_raw[1]:.1%} variance)')
ax.set_title('Colored by Disease Label\nDisease clusters don\'t align with variance')
ax.legend(title='Disease', fontsize=9)

plt.tight_layout()
plt.show()

print('Conclusion: The first principal component separates batches, not cases from controls.')
print('A naive analyst would discover Batch_01 enrichment as a "disease signal". We must fix this.')


---
## Step 1: Identity & Integrity — Sex Check and Duplicate Guard

Before any biological analysis, we verify that each sample is who it claims to be: correct sex metadata and a unique individual. The `xci_guard` and `sample_audit` modules handle this automatically.


### 1a — XCI Sex Guard: The Lab-Swap Check

In females (XX), one X chromosome is silenced (X-Chromosome Inactivation, XCI), producing a characteristic mean X-linked methylation of ~0.5. Males (XY) show low X methylation (~0.25). A mismatch between reported sex and observed X-linked signal reveals a sample swap or metadata error — which could be a case/control label flip in disguise.


In [ ]:
# ── Run the XCI guard ──────────────────────────────────────────────────────
# Note: the pipeline runs this after Stage 1a+1b QC, but we demo on raw data
# to show the signal is detectable even before sample-level QC.
sex_flagged_ids, sex_report = detect_sex_mixups(df_raw)
xci_full = compute_xci_signal(df_raw)
xci_full = xci_full.merge(
    sex_report[['sample_id', 'xci_mismatch']], on='sample_id', how='left'
)

print(f'Samples with sex-signal mismatch: {len(sex_flagged_ids)}')
print(f'  Flagged sample IDs: {sex_flagged_ids}')
print()
print('XCI report for flagged samples:')
display(sex_report[sex_report['sample_id'].isin(sex_flagged_ids)]
        [['sample_id', 'sex', 'mean_x_beta', 'n_x_cpgs', 'xci_mismatch']])

# ── Plot: Mean X-chromosome beta by reported sex ───────────────────────────
fig, ax = plt.subplots(figsize=(9, 5))

# Background shading for expected ranges
ax.axhspan(0.40, 0.65, alpha=0.10, color='#E91E63', label='Expected female range [0.40, 0.65]')
ax.axhspan(0.00, 0.35, alpha=0.10, color='#3498DB', label='Expected male range [0.00, 0.35]')

# Points colored by reported sex
for sex_label in ['M', 'F']:
    grp = xci_full[xci_full['sex'] == sex_label]
    ax.scatter(grp.index, grp['mean_x_beta'], color=SEX_PAL[sex_label],
               s=60, alpha=0.75, zorder=3, label=f'Reported {sex_label}')
    for _, row in grp.iterrows():
        ax.annotate(row['sample_id'], (row.name, row['mean_x_beta']),
                    textcoords='offset points', xytext=(3, 2), fontsize=5, alpha=0.5)

# Highlight mismatches
flagged_xci = xci_full[xci_full['sample_id'].isin(sex_flagged_ids)]
ax.scatter(flagged_xci.index, flagged_xci['mean_x_beta'],
           s=200, marker='X', color='red', zorder=6, linewidths=0,
           label='MISMATCH — flagged for removal')

ax.set_xlabel('Sample index', fontsize=11)
ax.set_ylabel('Mean X-chromosome beta value', fontsize=11)
ax.set_title(
    'XCI Sex Guard — Reported vs. Observed Sex Signal\n'
    'Samples outside their expected range are lab swaps or metadata errors',
    fontweight='bold'
)
ax.legend(fontsize=8, loc='upper right')
plt.tight_layout()
plt.show()

print('Interpretation:')
for sid in sex_flagged_ids:
    row = sex_report.set_index('sample_id').loc[sid]
    print(f'  {sid}: reported sex={row["sex"]}, mean_x_beta={row["mean_x_beta"]:.3f}  '
          f'-> signal inconsistent with reported sex -> REMOVED')


### 1b — Technical Duplicate Detection

Using pairwise Pearson correlation across the top 100 most variable CpGs, `sample_audit` identifies samples that are suspiciously identical. A pair with r > 0.99 is almost certainly the same library sequenced twice — keeping both would double-count one individual.


In [ ]:
# ── Run duplicate detection on post-QC data ────────────────────────────────
# To mirror the pipeline, run dedup after Stage 1a+1b+1c QC.
qc_pass     = audit_quality(df_raw)
contaminated, _ = detect_contamination(df_raw)
after_1ab   = [s for s in qc_pass if s not in contaminated]
df_1ab      = df_raw[df_raw['sample_id'].isin(after_1ab)].copy()
after_1abc  = [s for s in after_1ab if s not in sex_flagged_ids]
df_1abc     = df_raw[df_raw['sample_id'].isin(after_1abc)].copy()

dup_pairs, dups_to_drop = detect_duplicates(df_1abc)
flagged_pairs = dup_pairs[dup_pairs['duplicate_flag']]

print(f'Duplicate pairs detected: {len(flagged_pairs)}')
print(f'Samples to drop         : {dups_to_drop}')
print()
display(flagged_pairs[['sample_a', 'sample_b', 'pearson_r', 'duplicate_flag']])

# ── Correlation scatter for the flagged pair ───────────────────────────────
if len(flagged_pairs) > 0:
    pair = flagged_pairs.iloc[0]
    sid_a, sid_b = pair['sample_a'], pair['sample_b']

    beta_a = df_raw[df_raw['sample_id'] == sid_a][['cpg_id', 'beta_value']].rename(columns={'beta_value': sid_a})
    beta_b = df_raw[df_raw['sample_id'] == sid_b][['cpg_id', 'beta_value']].rename(columns={'beta_value': sid_b})
    merged = beta_a.merge(beta_b, on='cpg_id').sample(min(800, len(beta_a)), random_state=42)

    fig, ax = plt.subplots(figsize=(6, 6))
    ax.scatter(merged[sid_a], merged[sid_b], alpha=0.3, s=10, color='#9B59B6')
    ax.plot([0, 1], [0, 1], 'r--', linewidth=1.8, label=f'Identity line  (r = {pair["pearson_r"]:.6f})')
    ax.set_xlim(0, 1); ax.set_ylim(0, 1)
    ax.set_xlabel(f'{sid_a} beta value', fontsize=11)
    ax.set_ylabel(f'{sid_b} beta value', fontsize=11)
    ax.set_title(
        f'Technical Duplicate: {sid_a} vs {sid_b}\n'
        f'Pearson r = {pair["pearson_r"]:.6f}  —  {sid_b} will be removed',
        fontweight='bold'
    )
    ax.legend(fontsize=9)
    plt.tight_layout()
    plt.show()


### 1c — Applying All Quality-Control Exclusions

We now combine every exclusion list from Stages 1a–2 into a final clean cohort, then apply the site-level depth filter (Stage 2.5) to remove low-coverage CpG rows.


In [ ]:
# ── Build clean_samples (mirrors pipeline Stages 1a → 2) ──────────────────
all_samples    = df_raw['sample_id'].unique().tolist()
n_total        = len(all_samples)

qc_fail        = [s for s in all_samples if s not in qc_pass]
n_qc_failed    = len(qc_fail)
n_contaminated = len([s for s in contaminated if s in qc_pass])
n_sex_flagged  = len(sex_flagged_ids)
n_deduped      = len(dups_to_drop)

clean_samples  = [s for s in after_1abc if s not in dups_to_drop]
n_clean        = len(clean_samples)

df_clean = df_raw[df_raw['sample_id'].isin(clean_samples)].copy()

# ── Stage 2.5: site-level depth filter ────────────────────────────────────
df_clean, site_stats = filter_site_quality(df_clean, min_depth=5)

print('Quality-Control Exclusion Summary')
print('=' * 45)
print(f'  Input samples              : {n_total}')
print(f'  Bisulfite / depth failures : -{n_qc_failed}')
print(f'  Contamination flagged      : -{n_contaminated}')
print(f'  Sex metadata mixups (XCI)  : -{n_sex_flagged}')
print(f'  Technical duplicates       : -{n_deduped}')
print(f'  ─────────────────────────────────────────')
print(f'  Final clean samples        : {n_clean}')
print(f'  Site-level rows removed    : {site_stats["n_low"]:,} ({site_stats["pct_low"]:.1f}% < 5x depth)')
print(f'  Rows remaining in df_clean : {len(df_clean):,}')

# Exclusion waterfall bar
steps  = ['Start', 'Bisulfite\n/Depth', 'Contamination', 'Sex\nMixup', 'Duplicate', 'Clean']
counts = [n_total,
          n_total - n_qc_failed,
          n_total - n_qc_failed - n_contaminated,
          n_total - n_qc_failed - n_contaminated - n_sex_flagged,
          n_clean,
          n_clean]

fig, ax = plt.subplots(figsize=(10, 4))
bar_colors = ['#4A90D9'] + ['#F5A623'] * 4 + ['#27AE60']
bars = ax.bar(steps, counts, color=bar_colors, alpha=0.85, edgecolor='white', linewidth=0.5)
for bar, cnt in zip(bars, counts):
    ax.text(bar.get_x() + bar.get_width() / 2, bar.get_height() + 0.3, str(cnt),
            ha='center', va='bottom', fontsize=10, fontweight='bold')
ax.set_ylabel('Number of samples')
ax.set_title('Sample Exclusion Waterfall — All QC Stages', fontweight='bold')
ax.set_ylim(0, n_total + 4)
plt.tight_layout()
plt.show()


---
## Step 2: Surgical Strike — Clonal VDJ Masking

Some Case samples show extremely high methylation (beta > 0.8) at VDJ recombination loci with unusually long fragment lengths (> 180 bp). This is not a real methylation signal — it is a clonal expansion artifact where a dominant B-cell or T-cell clone has flooded the library with its VDJ-rearranged genome.

The correct treatment is surgical: we keep the sample (it contains valid signal elsewhere) but mask the specific VDJ CpGs that are contaminated by the artifact.


### 2a — The Problem: A "Fake Biomarker" in the VDJ Locus


In [ ]:
# ── Capture pre-site-filter snapshot for demo visualization ───────────────
# Site filtering (Stage 2.5) removes rows with depth < 5 and could trim
# VDJ artifact rows before flag_clonal_artifacts sees them. To guarantee the
# artifact is always visible in this walkthrough, detection and visualization
# run on the full clean-sample data BEFORE the depth filter.
# The masking in Step 2b still operates on df_clean (site-filtered) — correct
# pipeline behavior. Masking skips rows that were already dropped by the filter.
df_pre_site = df_raw[df_raw['sample_id'].isin(clean_samples)].copy()

# ── Clonal detection on pre-site-filter data (authoritative for demo) ──────
clonal_rows, clonal_samples = flag_clonal_artifacts(df_pre_site)

# Cross-check: confirm site filter did not silently hide the artifact
_, clonal_post = flag_clonal_artifacts(df_clean)
if set(clonal_samples) != set(clonal_post):
    print('Note: site filter altered VDJ artifact visibility.')
    print(f'  Pre-filter detected : {clonal_samples}')
    print(f'  Post-filter detected: {clonal_post}')
    print('  Using pre-filter result for demo visualization.')
    print('  Step 2b masking is applied to df_clean with this clonal_samples list.')
else:
    print('Clonal detection is consistent before and after the site depth filter.')

print(f'Clonal samples : {clonal_samples}')
print(f'Clonal VDJ rows: {len(clonal_rows)}')
if len(clonal_rows):
    print()
    print('Artifact statistics per clonal sample:')
    stats = clonal_rows.groupby('sample_id').agg(
        n_rows=('beta_value', 'count'),
        mean_beta=('beta_value', 'mean'),
        mean_frag=('fragment_length', 'mean'),
    ).round(3)
    display(stats)

# ── VDJ profile scatter — pre-site-filter data ensures artifact is visible ─
vdj_viz  = df_pre_site[df_pre_site['is_vdj_region']].copy()
nvdj_viz = df_pre_site[~df_pre_site['is_vdj_region']].sample(
    min(500, len(df_pre_site[~df_pre_site['is_vdj_region']])), random_state=42)

vdj_viz['is_clonal'] = vdj_viz['sample_id'].isin(clonal_samples)

fig, ax = plt.subplots(figsize=(9, 6))

ax.scatter(nvdj_viz['fragment_length'], nvdj_viz['beta_value'],
           alpha=0.12, s=8, color='#CCCCCC', label='Non-VDJ sites (subsample)', zorder=1)

normal_vdj = vdj_viz[~vdj_viz['is_clonal']]
ax.scatter(normal_vdj['fragment_length'], normal_vdj['beta_value'],
           alpha=0.40, s=18, color='#4A90D9', label='VDJ sites (normal samples)', zorder=2)

clonal_vdj = vdj_viz[vdj_viz['is_clonal']]
ax.scatter(clonal_vdj['fragment_length'], clonal_vdj['beta_value'],
           alpha=0.85, s=55, color='#E05C5C',
           label=f'VDJ sites — CLONAL {clonal_samples}', zorder=3)

ax.axhline(0.80, color='#8B0000', linestyle='--', linewidth=1.5, alpha=0.85,
           label='beta > 0.80 threshold')
ax.axvline(180,  color='#8B0000', linestyle=':',  linewidth=1.5, alpha=0.85,
           label='frag > 180 bp threshold')
x_max = max(int(df_pre_site['fragment_length'].max()), 350)
ax.fill_between([180, x_max], 0.80, 1.0, alpha=0.06, color='red')
ax.text(185, 0.91, 'Fake-biomarker zone\nbeta > 0.8  AND  frag > 180 bp',
        color='#8B0000', fontsize=8, va='top')

ax.set_xlabel('Fragment length (bp)', fontsize=11)
ax.set_ylabel('Beta value', fontsize=11)
ax.set_title(
    'VDJ Clonal Profile — Beta vs Fragment Length\n'
    'The clonal sample occupies the top-right "fake biomarker" zone',
    fontweight='bold',
)
ax.legend(fontsize=8)
ax.set_xlim(left=50)
ax.set_ylim(0, 1)
plt.tight_layout()
plt.show()


### 2b — The Fix: Flag and Mask the Clonal Signal

`mask_clonal_vdj_sites()` sets `beta_value = NaN` only at VDJ-locus rows for clonal samples. Every other CpG from that sample is preserved. This "surgical" approach saves the sample's non-VDJ biology while neutralizing the artifact.


In [ ]:
# Save df_clean snapshot before masking (for the heatmap)
df_before_mask = df_clean.copy()

# ── Apply VDJ masking (Stage 3.5) ─────────────────────────────────────────
if clonal_samples:
    df_masked, n_masked = mask_clonal_vdj_sites(df_clean, clonal_samples)
    print(f'VDJ sites masked (beta -> NaN) : {n_masked} rows')
    print(f'In sample(s)                   : {clonal_samples}')

    # Verify: clonal sample VDJ sites now have NaN beta values
    clonal_vdj_after = df_masked[
        (df_masked['sample_id'].isin(clonal_samples)) &
        (df_masked['is_vdj_region'])
    ]
    pct_nan = clonal_vdj_after['beta_value'].isna().mean() * 100
    print(f'Verification: {pct_nan:.1f}% of clonal VDJ rows now have beta_value = NaN')

    # Non-VDJ rows for clonal sample are untouched
    clonal_nvdj = df_masked[
        (df_masked['sample_id'].isin(clonal_samples)) &
        (~df_masked['is_vdj_region'])
    ]
    pct_ok = clonal_nvdj['beta_value'].notna().mean() * 100
    print(f'Non-VDJ rows for clonal sample  : {pct_ok:.1f}% intact (no masking applied)')
else:
    df_masked = df_clean.copy()
    n_masked  = 0
    print('No clonal samples detected; df_masked = df_clean')


### 2c — The Surgical Heatmap: Before and After

The heatmap shows 10 VDJ CpGs × 5 samples (one clonal, four normal). Before masking, the clonal sample shows bright-red high-beta values at these sites — the "fake biomarker". After masking, those same cells are `NaN` (shown in gray), and downstream models will impute them to approximately zero (the cohort mean after median centering).


In [ ]:
if clonal_samples:
    clonal_sid = clonal_samples[0]

    # ── Select 10 VDJ CpGs from df_pre_site (full artifact view) ─────────────
    # Using pre-site-filter data ensures we always have CpGs to show, even if
    # the site depth filter removed borderline-depth VDJ rows from df_clean.
    clonal_vdj_cpgs_all = sorted(
        df_pre_site[
            (df_pre_site['sample_id'] == clonal_sid) &
            (df_pre_site['is_vdj_region'])
        ]['cpg_id'].unique()
    )
    heatmap_cpgs = clonal_vdj_cpgs_all[:10]

    # Pick 4 normal samples (clean, not clonal)
    normal_sids  = sorted([s for s in clean_samples if s not in clonal_samples])[:4]
    heatmap_sids = [clonal_sid] + normal_sids

    # ── BEFORE pivot: pre-site-filter raw values ───────────────────────────────
    # Shows the artifact at full strength — the unmasked, unfiltered inject.
    before_pivot = (
        df_pre_site[
            df_pre_site['cpg_id'].isin(heatmap_cpgs) &
            df_pre_site['sample_id'].isin(heatmap_sids)
        ]
        .pivot(index='cpg_id', columns='sample_id', values='beta_value')
        .reindex(columns=heatmap_sids, index=heatmap_cpgs)
    )

    # ── AFTER pivot: pipeline-masked values ───────────────────────────────────
    # df_masked = df_clean (site-filtered) with clonal VDJ rows set to NaN.
    # CpGs absent here were removed by site filter before masking.
    after_pivot = (
        df_masked[
            df_masked['cpg_id'].isin(heatmap_cpgs) &
            df_masked['sample_id'].isin(heatmap_sids)
        ]
        .pivot(index='cpg_id', columns='sample_id', values='beta_value')
        .reindex(columns=heatmap_sids, index=heatmap_cpgs)
    )

    # ── Side-by-side heatmaps ──────────────────────────────────────────────────
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    fig.suptitle(
        f'Surgical Heatmap — VDJ CpGs: Clonal Sample ({clonal_sid}) vs 4 Normal Samples',
        fontsize=12, fontweight='bold',
    )

    col_labels = [f'{s}\n(CLONAL)' if s == clonal_sid else s for s in heatmap_sids]

    sns.heatmap(before_pivot, ax=axes[0], vmin=0, vmax=1,
                cmap='RdYlBu_r', annot=True, fmt='.2f', linewidths=0.5,
                cbar_kws={'label': 'Beta value'},
                xticklabels=col_labels)
    axes[0].set_title(
        'BEFORE Masking  (raw, pre-site-filter)\nClonal column: inflated beta 0.8–1.0',
        fontweight='bold')
    axes[0].set_ylabel('CpG site')
    axes[0].tick_params(axis='x', labelsize=8)

    sns.heatmap(after_pivot, ax=axes[1], vmin=0, vmax=1,
                cmap='RdYlBu_r', annot=True, fmt='.2f', linewidths=0.5,
                cbar_kws={'label': 'Beta value'},
                xticklabels=col_labels)
    # Annotate cells that are NaN in the after pivot
    for cpg_idx, cpg in enumerate(heatmap_cpgs):
        for sid_idx, sid in enumerate(heatmap_sids):
            val = after_pivot.loc[cpg, sid] if cpg in after_pivot.index else float('nan')
            if pd.isna(val):
                label = 'NaN\n(masked)' if cpg in before_pivot.index and not pd.isna(before_pivot.loc[cpg, sid]) else 'N/A\n(filtered)'
                axes[1].text(sid_idx + 0.5, cpg_idx + 0.5, label,
                             ha='center', va='center', fontsize=7,
                             color='black', fontweight='bold')

    axes[1].set_title(
        'AFTER Masking  (site-filtered + masked)\nClonal VDJ cells -> NaN -> imputed ~0',
        fontweight='bold')
    axes[1].set_ylabel('')
    axes[1].tick_params(axis='x', labelsize=8)

    plt.tight_layout()
    plt.show()

    print()
    print('Result: The clonal sample is RETAINED in the cohort.')
    print('Only its VDJ CpG rows are neutralized. All other signal is preserved.')
    print('Downstream models (DMR hunter, ML guard) impute the NaN rows to ~0.')
else:
    print('No clonal samples detected; heatmap skipped.')


---
## Step 3: Harmonization — Batch Normalization

Even after removing all bad samples, we showed in Step 0 that `Batch_01` is enriched for Case samples (Cramér's V > 0.5). The `normalizer` module checks for this confound and applies median centering — a method that preserves relative CpG-to-CpG differences within a sample while removing the batch-driven offset.

> Why not quantile normalization? It distorts the bimodal shape of methylation distributions, which is a biologically meaningful feature. Median centering does not.


In [ ]:
# ── Stage 4: Confound check ────────────────────────────────────────────────
confound = check_confounding(df_masked, 'batch_id', 'disease_label')

print('Batch x Disease Confound Check (Cramers V)')
print('=' * 45)
print(f"  Cramer's V  : {confound['cramers_v']:.4f}")
print(f"  chi2        : {confound['chi2']:.2f}  (p = {confound['p_value']:.4e})")
print(f"  Confounded? : {'YES — V > 0.50; batch correction is biased' if confound['confounded'] else 'No'}")
print()
print('Contingency table (batch x disease):')
display(confound['contingency_table'])

# ── Stage 4: Normalize ────────────────────────────────────────────────────
df_norm = robust_normalize(df_masked, save_figure=False)

print()
print(f"Normalization complete.  'beta_normalized' column added.")
print(f"Columns now: {list(df_norm.columns)}")

# ── Before / After PCA ────────────────────────────────────────────────────
pca_before, var_before = _run_pca(df_masked, value_col='beta_value')
pca_after,  var_after  = _run_pca(df_norm,   value_col='beta_normalized')

fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('Normalization Effect — Before vs After PCA', fontsize=13, fontweight='bold')

# ── Left: Before normalization, colored by batch ───────────────────────────
ax = axes[0]
for batch, grp in pca_before.groupby('batch_id'):
    ax.scatter(grp['PC1'], grp['PC2'], label=batch,
               color=BATCH_PAL.get(batch, 'gray'), alpha=0.85, s=70, edgecolors='white', linewidths=0.5)
ax.set_xlabel(f'PC1 ({var_before[0]:.1%} variance)')
ax.set_ylabel(f'PC2 ({var_before[1]:.1%} variance)')
ax.set_title('BEFORE Normalization\nColored by Batch (problem visible)', fontweight='bold')
ax.legend(title='Batch', fontsize=9)

# ── Right: After normalization, colored by disease ─────────────────────────
ax = axes[1]
for disease, grp in pca_after.groupby('disease_label'):
    ax.scatter(grp['PC1'], grp['PC2'], label=disease,
               color=DISEASE_PAL.get(disease, 'gray'), alpha=0.85, s=70, edgecolors='white', linewidths=0.5)
ax.set_xlabel(f'PC1 ({var_after[0]:.1%} variance)')
ax.set_ylabel(f'PC2 ({var_after[1]:.1%} variance)')
ax.set_title('AFTER Normalization\nColored by Disease (batch clusters dissolved)', fontweight='bold')
ax.legend(title='Disease', fontsize=9)

plt.tight_layout()
plt.show()

print()
print('After median centering, the dominant batch-driven principal component is removed.')
print('Variance now reflects biological signal (disease label) rather than technical noise.')


---
### Visual 4 — Covariate Structure: Batch, Disease Label, Sex, Age

PC1 separates Batch_01 Cases from everyone else — driven by the batch confound and the true biological DMR signal acting together. PC2 separates males from females within the Batch_01 Case cluster, with zero overlap (mean PC2: F = +0.79, M = −0.69).

This confirms that both sex and age must be included as covariates in any linear model applied to this cohort. A naive Wilcoxon test comparing Cases vs. Controls without adjusting for sex risks flagging sex-dimorphic autosomal CpGs as disease signal.

In [ ]:
from core.visuals import plot_pca_covariates
from IPython.display import Image

cov_path = plot_pca_covariates(df_norm)
Image(cov_path)

---
## Step 4: Scientific Discovery — Differentially Methylated Regions

With a clean, normalized dataset, we run `dmr_hunter`: a sliding-window Wilcoxon test that compares Case vs. Control methylation across each 5-CpG window. Windows passing both p_adj < 0.05 and |ΔBeta| > 0.10 are called significant.

The true biological signal — 11 CpGs (cg00000300–cg00000310) with a +0.25 Case shift — was injected at data-generation time. Can we recover it?


In [ ]:
# ── Stage 6: DMR Hunter ────────────────────────────────────────────────────
dmrs = find_dmrs(df_norm, clean_samples, normalized_col='beta_normalized')

sig_dmrs    = dmrs[dmrs['significant']]
clonal_dmrs = dmrs[dmrs['clonal_risk']]
sig_clonal  = sig_dmrs[sig_dmrs['clonal_risk']]

print(f'Total windows tested     : {len(dmrs):,}')
print(f'Significant DMRs (p_adj < 0.05, |dBeta| > 0.10) : {len(sig_dmrs)}')
print(f'Windows overlapping VDJ loci (clonal_risk=True)  : {len(clonal_dmrs)}')
print(f'Sig DMRs with clonal risk                        : {len(sig_clonal)}')
print()
print('Significant DMRs:')
display(sig_dmrs[['window_id', 'n_cpgs', 'case_mean', 'ctrl_mean',
                   'delta_beta', 'p_value', 'p_adj', 'n_vdj_cpgs', 'clonal_risk']]
        .sort_values('p_adj')
        .style.background_gradient(subset=['delta_beta'], cmap='RdBu_r')
        .format({'case_mean': '{:.3f}', 'ctrl_mean': '{:.3f}',
                 'delta_beta': '{:+.3f}', 'p_value': '{:.2e}', 'p_adj': '{:.2e}'}))


In [ ]:
# ── Volcano plot with highlighted signal and VDJ annotation ───────────────
fig, axes = plt.subplots(1, 2, figsize=(15, 6))
fig.suptitle('DMR Volcano Plots — From Noise to Signal', fontsize=13, fontweight='bold')

for ax_idx, (ax, title_suffix) in enumerate(zip(axes, ['Standard', 'Annotated: VDJ windows in orange'])):

    # All non-significant windows
    ns = dmrs[~dmrs['significant'] & ~dmrs['clonal_risk']]
    ax.scatter(ns['delta_beta'], -np.log10(ns['p_adj'] + 1e-10),
               alpha=0.25, s=10, color='#AAAAAA', label='Not significant')

    if ax_idx == 1:
        # VDJ / clonal-risk windows in orange
        cr = dmrs[dmrs['clonal_risk'] & ~dmrs['significant']]
        ax.scatter(cr['delta_beta'], -np.log10(cr['p_adj'] + 1e-10),
                   alpha=0.6, s=30, color='#F5A623', label='VDJ clonal-risk window', zorder=3)
        cr_sig = dmrs[dmrs['clonal_risk'] & dmrs['significant']]
        if len(cr_sig):
            ax.scatter(cr_sig['delta_beta'], -np.log10(cr_sig['p_adj'] + 1e-10),
                       alpha=0.9, s=60, color='#F5A623', edgecolors='black', linewidths=0.8,
                       label='VDJ sig. DMR (treat with caution)', zorder=4)

    # Non-VDJ significant windows
    sig_nonvdj = sig_dmrs[~sig_dmrs['clonal_risk']]
    ax.scatter(sig_nonvdj['delta_beta'], -np.log10(sig_nonvdj['p_adj'] + 1e-10),
               alpha=0.9, s=80, color='#E05C5C', edgecolors='black', linewidths=0.8,
               label='Significant DMR (true biology)', zorder=5)

    # Annotate significant DMRs
    for _, row in sig_dmrs.iterrows():
        ax.annotate(
            f"{row['window_id']}\ndBeta={row['delta_beta']:+.2f}",
            (row['delta_beta'], -np.log10(row['p_adj'] + 1e-10)),
            textcoords='offset points', xytext=(6, 4),
            fontsize=7, color='#8B0000', fontweight='bold'
        )

    # Threshold lines
    ax.axhline(-np.log10(0.05), color='gray', linestyle='--', linewidth=0.9, label='p_adj = 0.05')
    ax.axvline( 0.10, color='gray', linestyle=':', linewidth=0.9)
    ax.axvline(-0.10, color='gray', linestyle=':', linewidth=0.9)

    ax.set_xlabel('Delta Beta (Case - Control)', fontsize=11)
    ax.set_ylabel('-log10(p_adj)', fontsize=11)
    ax.set_title(title_suffix, fontweight='bold')
    ax.legend(fontsize=7, loc='upper left')

plt.tight_layout()
plt.show()

if len(sig_dmrs):
    top = sig_dmrs.sort_values('p_adj').iloc[0]
    print(f'Top hit: {top["window_id"]}  '
          f'dBeta={top["delta_beta"]:+.3f}  '
          f'p_adj={top["p_adj"]:.2e}  '
          f'clonal_risk={top["clonal_risk"]}')
    if not top['clonal_risk']:
        print('The top DMR carries NO clonal risk flag — it reflects true biology, not VDJ artifact.')


---
## Step 5: Machine Learning Validation — ML Guard

The final validation step asks: can the cleaned signal actually discriminate Case from Control? We use an ElasticNet logistic classifier with `GroupKFold` cross-validation — leakage-proof because samples from the same patient never appear in both train and test folds.

The key narrative: Without VDJ masking, a model could exploit the clonal methylation artifact to achieve an artificially inflated AUC. By masking those sites _before_ the model ever sees them, the AUC reflects only genuine disease-associated CpGs.


In [ ]:
# ── Stage 7: ML Guard ─────────────────────────────────────────────────────
print('Running ElasticNet + GroupKFold CV on the cleaned, normalized data...')
ml_result = run_safe_model(df_norm, feature_col='beta_normalized')

print()
print('ML Guard Results')
print('=' * 45)
print(f"  Mean AUC         : {ml_result['mean_auc']:.4f} +/- {ml_result['std_auc']:.4f}")
print(f"  Mean Accuracy    : {ml_result['mean_accuracy']:.4f}")
print(f"  n_samples        : {ml_result['n_samples']}")
print(f"  n_features (CpGs): {ml_result['n_features']}")
if ml_result.get('warning'):
    print(f"  Warning          : {ml_result['warning']}")

print()
print('Cross-validation fold detail:')
for fold_key, fold_val in ml_result['cv_results'].items():
    print(f'  {fold_key}: {fold_val}')

# ── AUC bar chart with narrative ───────────────────────────────────────────
fig, ax = plt.subplots(figsize=(8, 5))

auc_val  = ml_result['mean_auc']
auc_std  = ml_result['std_auc']
bar_col  = '#27AE60' if auc_val >= 0.70 else '#E05C5C'

ax.barh(['Honest AUC\n(VDJ masked)'], [auc_val], color=bar_col, alpha=0.85, height=0.35,
        xerr=[auc_std], error_kw={'elinewidth': 2, 'capsize': 6, 'capthick': 2, 'ecolor': 'black'})
ax.axvline(0.5,  color='red',  linestyle='--', linewidth=1.2, label='Random classifier (0.50)')
ax.axvline(0.70, color='gray', linestyle=':',  linewidth=1.0, label='Clinical relevance threshold (0.70)')
ax.text(auc_val + auc_std + 0.01, 0, f'{auc_val:.4f}', va='center', fontsize=11, fontweight='bold', color=bar_col)

ax.set_xlim(0, 1.1)
ax.set_xlabel('ROC-AUC', fontsize=11)
ax.set_title(
    'ElasticNet + GroupKFold CV — Classification AUC\n'
    'VDJ clonal sites are masked; AUC reflects genuine disease signal',
    fontweight='bold'
)
ax.legend(fontsize=9)
plt.tight_layout()
plt.show()

print()
print('Interpretation:')
if auc_val >= 0.95:
    print(f'  AUC = {auc_val:.4f}: The injected biological signal (11 CpGs, +0.25 dBeta) is')
    print('  strong enough for near-perfect separation in this mock dataset.')
    print('  In real clinical data with a weaker signal-to-noise ratio, an honest')
    print('  AUC of 0.70-0.90 is typical after proper artifact removal.')
print()
print('Key guarantee: GroupKFold ensures no patient appears in both train and test.')
print('The AUC cannot be inflated by sample-level data leakage.')


---
## Summary: From Noise to Signal

This notebook walked through a complete, rigorous artifact-removal pipeline applied to simulated B-cell/T-cell DNA methylation data.

| Stage | Action | Samples / Rows Removed |
|-------|--------|------------------------|
| 1a: Bisulfite QC | Non-CpG rate > 2% or mean depth < 10x | 3 samples |
| 1b: Contamination | Bimodal beta distribution (S020) | 1 sample |
| 1c: XCI Guard | X-linked beta contradicts reported sex | 2 samples |
| 2: Deduplication | Pearson r > 0.99 (S010 ↔ S_DUP) | 1 sample |
| 2.5: Site Filter | CpG rows with depth < 5x | variable rows |
| 3: Clonal Masking | VDJ beta > 0.8 AND fragment > 180 bp | NaN at VDJ rows only |
| 4: Normalization | Median centering per sample | (batch offset removed) |

Final result: 41 → 34 clean samples, 1 significant DMR (w00305, ΔBeta ≈ +0.185, p_adj ≈ 1.2e-05), AUC reflecting genuine disease-associated methylation.

---

> Every detection threshold, statistical test, and correction method demonstrated in this notebook lives in `core/`. The notebook itself is a display layer only — reproducible by re-running all cells from top to bottom.
